In [34]:
import numpy as np
import gradio as gr
import requests
from PIL import Image
from openai import OpenAI


def run_demo():
    """Setup the app interface and launch it."""
    with gr.Blocks() as app:

        gr.Markdown('# Mental Health Nudging with Generative AI Demo')
        with gr.Row():

            # input features
            with gr.Column(scale=1):

                # demographics
                gender = gr.Radio(label='Gender', choices=['Male', 'Female', 'Non-Binary'])
                age = gr.Slider(label='Age', minimum=18, maximum=80, step=1)
                race = gr.Radio(label='Race', choices=['white', 'hispanic', 'black', 'asian', 'unspecified'])

                # symptoms
                #symptoms = gr.CheckboxGroup(label='Diagnosed Mental Health Conditions',
                #                            choices=['Depression', 'Generalized anxiety disorder (GAD)', 'Panic disorder', 'Social anxiety disorder', 'Obsessive-compulsive disorder (OCD)', 'Post-traumatic stress disorder (PTSD)'])

                # symptoms
                symptoms = gr.CheckboxGroup(label='Symptoms',
                                            choices=['sadness', 'inability to concentrate', 'excessive worrying', 'extreme mood changes', 'withdrawal from friends/activities', 'tiredness', 'hallucinations', 'addiction', 'lack of appetite', 'increased appetite',])

                # interests
                likes = gr.Textbox(label='Liked Activities',
                                   placeholder='Comma-separated list of likes...')
                dislikes = gr.Textbox(label='Disliked Activities',
                                      placeholder='Comma-separated list of dislikes...')

                # submit button
                submit_button = gr.Button('Generate Nudge')

            # resulting nudge
            with gr.Column(scale=1):
                nudge_image = gr.Image(label='Nudge Image')
                nudge_message = gr.Textbox(label='Nudge Message')

        # submit parameters for nudge generation
        inputs = [gender, age, race, symptoms, likes, dislikes]
        outputs = [nudge_image, nudge_message]
        submit_button.click(fn=generate, inputs=inputs, outputs=outputs)

    # launch the app
    gr.close_all()
    app.queue(default_concurrency_limit=None)
    app.launch()


def generate(gender, age, race, symptoms, likes, dislikes):
    """Generate nudging image and message for the given person."""
    # construct description of the person
    if race in ['white', 'hispanic', 'black', 'asian']:
        race_desc = f'{race.lower()}'
    else:
        race_desc = ''
    desc = f'A {age} year old {"man" if gender=="Male" else "woman" if gender=="Female" else "non-binary person"}.'
    if likes:
        desc += f' They like {likes}.'
    if dislikes:
        desc += f' They do not like {dislikes}.'
    if symptoms:
        desc += f' They have the following mental health symptoms: {", ".join(symptoms)}.'
    else:
        desc += f' They do not have any mental health symptoms.'

    # generate nudge for the person
    nudge_message = generate_nudge_message(desc)
    nudge_image = generate_nudge_image(age, gender, race_desc, desc, nudge_message, random=False)

    return nudge_image, nudge_message


def generate_nudge_message(desc):
    """Generate a message for a given person."""
    print(f'{desc=}')
    system_prompt = '''You are writing motivational text messages to help people with their mental health. \
    Messages should be friendly and positive, but also professional and super short. \
    You are limited on space. \
    Messages should be written at the reading level of an eighth grader. \
    Word choice should be short and simple so everyone can understand. \
    
    You will be given some basic information about the person you are addressing. \
    DO NOT reference all of their likes if there are more than two. Be discerning. \
    You should try to use the person's information to give them relevant and actionable tips for improving their mental health symptoms.'''
    user_prompt = f'Write a short inspirational message for the person with the following description:\n\n{desc}'
    messages = [{'role': 'system', 'content': f'{system_prompt}'},
                {'role': 'user', 'content': f'{user_prompt}'}]
    completion = client.chat.completions.create(messages=messages, model='gpt-3.5-turbo', temperature=.5)
    nudge_message = completion.choices[0].message.content
    return nudge_message


def generate_nudge_image(age, gender, race_desc, desc, nudge_message, random=True):
    """Generate an image for a given person and message."""
    if random:
        return Image.fromarray(np.random.randint(0, 255, (100, 100, 3), dtype='uint8'), 'RGB')

    prompt = 'Illustrate one simple, inspirational, fun image to help a person with their mental health. Do not include text. '\
           + f'The style is cute and illustrative. '\
           + f'The person is a {age} year old {race_desc} {"man" if gender=="Male" else "woman" if gender=="Female" else "non-binary person"}. '\
           + f'The image should align with the following message:\n\n{nudge_message}'

    print(f'{prompt=}')
    response = client.images.generate(prompt=prompt, model='dall-e-3', style='vivid', quality='standard')
    nudge_image = Image.open(requests.get(response.data[0].url, stream=True).raw)

    return nudge_image


if __name__ == '__main__':
    client = OpenAI()
    run_demo()


Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.


desc='A 29 year old non-binary person. They like cats. They have the following mental health symptoms: sadness.'
prompt="Illustrate one simple, inspirational, fun image to help a person with their mental health. Do not include text. The style is cute and illustrative. The person is a 29 year old white non-binary person. The image should align with the following message:\n\nHey there! 🌟 Remember, you're not alone in this journey. Take small steps to brighten your day. Spend time with your furry friend, cuddle up with your cat. Their purrs can bring comfort and joy. Embrace the simple pleasures in life and let them bring a smile to your face. You've got this! 💪❤️"
desc='A 52 year old non-binary person. They like dogs. They have the following mental health symptoms: tiredness.'
prompt='Illustrate one simple, inspirational, fun image to help a person with their mental health. Do not include text. The style is cute and illustrative. The person is a 52 year old black non-binary person. The i